# Dating study analyses

## Prep

In [3]:
# packages (might not need all of these)

import os
import h5py
import pandas as pd
import numpy as np
from glob import glob
import nibabel as nib
import random
import json
import pickle
import warnings

from nilearn.plotting import view_img, glass_brain, plot_stat_map
from nilearn.image import load_img, smooth_img
from nilearn import plotting
from nilearn.reporting import get_clusters_table

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.spatial.distance import cdist

from sklearn import linear_model
from progressbar import ProgressBar

# custom functions

def parc_files(p):
    if 'fmriprep' in p:
        wholebrain = '/home/bms2202/dating/masks/resampled/group_fmriprep/wholebrain.nii.gz'
        parcel_file = '/home/bms2202/dating/masks/resampled/group_fmriprep/shen.nii.gz'
        denoised = 'denoised_' + p
        analyses = 'analyses_' + p
    elif 'shen' in p:
        wholebrain = '/home/bms2202/dating/masks/resampled/shen/shen.nii.gz'
        parcel_file = '/home/bms2202/dating/masks/resampled/shen/shen.nii.gz'
        denoised = 'denoised_' + p
        analyses = 'analyses_' + p
    elif 'surface' in p:
        parcel_file_folder = 'masks/original/yeo/FreeSurfer5.3/fsaverage6/label/'
        wholebrain = ['masks/freesurfer_surfaces/pial_left.gii.gz',
                     'masks/freesurfer_surfaces/pial_right.gii.gz',
                     'masks/freesurfer_surfaces/infl_left.gii.gz',
                     'masks/freesurfer_surfaces/infl_right.gii.gz']
        parcel_file = [parcel_file_folder + 'lh.Schaefer2018_200Parcels_17Networks_order.annot',
                      parcel_file_folder + 'rh.Schaefer2018_200Parcels_17Networks_order.annot']
        denoised = 'denoised_' + p
        analyses = 'analyses_' + p
    return denoised, analyses, wholebrain, parcel_file

# pick ROI
def pick_roi2(roi_name, wholebrain,parc_file):
    if 'surface' in denoised:
        if type(roi_name) == str:
            roi_file = 'masks/original/yeo/' + roi_name + '_verts.h5'
            roi_mask = h5py.File(roi_file,'r')
            roi_mask = roi_mask['data']['both'][:].astype(bool)
        else:
            roi_file_l = nib.freesurfer.io.read_annot(parc_file[0])
            roi_file_r = nib.freesurfer.io.read_annot(parc_file[1])
            roi_file = np.concatenate([roi_file_l[0],roi_file_r[0] + 101])
            roi_mask = np.isin(roi_file,roi_name)
    else:
        if type(roi_name) == str:
            roi_file = os.path.dirname(parc_file) + '/' + roi_name + '.nii.gz'
            roi_mask = nib.load(roi_file).get_fdata().astype(bool)
            wholebrain_mask = nib.load(wholebrain).get_fdata().astype(bool)
            roi_mask = roi_mask[wholebrain_mask]
        else:
            roi_file = parc_file
            roi_mask = nib.load(roi_file).get_fdata()
            wholebrain_mask = nib.load(wholebrain).get_fdata().astype(bool)
            roi_mask = roi_mask[wholebrain_mask]
            roi_mask = np.isin(roi_mask,roi_name)
    return roi_mask

# hack to use nilearn plotting tools
def create_plottable_image(parc_file,data,roi = None):
    maskf = nib.load(parc_file)
    data_3d = np.zeros(maskf.get_fdata().shape)
    if roi:
        bool_mask = np.isin(maskf.get_fdata(), roi)
    else:
        bool_mask = maskf.get_fdata().astype(bool)
    data_3d[bool_mask] = data
    return nib.Nifti2Image(data_3d, maskf.affine), data_3d

## Setting up ROIs

### Hippocampus

In [ ]:
roi_names = ['hippocampus']
roises = [[17, 53]]
pct50 = 150 # change this if I change number of subs or runs

subs = sorted(glob('fmriprep/sub-*'))
subs = [x.split('/')[1] for x in subs]
tasktypes = ['photos_run-0','photos_run-1','photos_run-2',
            'ratings_run-1','ratings_run-2',
            'rest_run-0','rest_run-1','rest_run-2',
            'recall_run-1','recall_run-2']
sample = nib.load('fmriprep/sub-DAT004/ses-OchsnerDating/func/sub-DAT004_ses-OchsnerDating_task-photos_run-0_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')

for dataversion in ['fmriprep']:
    denoised, analyses, wholebrain, parc_file = parc_files(dataversion)
    print(dataversion)
    group_mask = nib.load(wholebrain)
    group_maskf = group_mask.get_fdata().astype(bool)
    
    for i in range(len(roi_names)):
        roi_name = roi_names[i]
        rois = roises[i]

        all_arrays = np.zeros([len(tasktypes),len(subs),sample.shape[0],
                               sample.shape[1],sample.shape[2]])

        pbar = ProgressBar()
        for t, tasktype in enumerate(pbar(tasktypes)):
            for i,sub in enumerate(subs):
                bdata = nib.load(f'fmriprep/{sub}/ses-OchsnerDating/func/{sub}_ses-OchsnerDating_task-{tasktype}_space-MNI152NLin2009cAsym_desc-aparcaseg_dseg.nii.gz').get_fdata()
                bdata_new = np.where(np.isin(bdata,rois),1,0)
                all_arrays[t,i,:] = bdata_new
        
        roi_mask = all_arrays.sum(axis = 1).sum(axis = 0)
        roi_mask = np.where(roi_mask > pct50,1,0)
        
        #roi_group_mask = roi_mask[group_maskf] # make it 1D
        roi_group_mask = roi_mask*group_maskf # keep it 3D
        
        mask_nifti = nib.Nifti2Image(roi_group_mask.astype(np.float64),group_mask.affine)
        if dataversion == 'fmriprep':
            nib.save(mask_nifti,'masks/resampled/group_fmriprep/' + roi_name + '.nii.gz')

In [ ]:
# divide up the hippocampus into anterior and posterior

##### posterior
img = nib.load('masks/resampled/group_fmriprep/hippocampus.nii.gz')

## one hemi
mask = img.get_fdata()[:39,:,:]
aff = img.affine

mask_new = np.zeros(mask.shape, dtype = float)
ind = np.where(mask == 1)
indp = min(ind[1][:]) # check axis!
inda = max(ind[1][:]) # check axis!
third = int(np.floor((inda - indp) / 3))

mask_new[:, indp:indp + third*2 + 1, :] = 1 # check axis!
post1 = mask
post1[mask_new == 0] = 0

## other hemi
mask = img.get_fdata()[39:,:,:]
aff = img.affine

mask_new = np.zeros(mask.shape, dtype = float)
ind = np.where(mask == 1)
indp = min(ind[1][:]) # check axis!
inda = max(ind[1][:]) # check axis!
third = int(np.floor((inda - indp) / 3))

mask_new[:, indp:indp + third*2 + 1, :] = 1 # check axis!
post2 = mask
post2[mask_new == 0] = 0

post = np.concatenate((post1,post2))

# save ROI
img_post = nib.Nifti2Image(post, aff)
img_post.to_filename('masks/resampled/group_fmriprep/hippocampus_posterior.nii.gz')
img_post = nib.Nifti2Image(post1, aff)
img_post.to_filename('masks/resampled/group_fmriprep/hippocampus_posteriorL.nii.gz')
img_post = nib.Nifti2Image(post2, aff)
img_post.to_filename('masks/resampled/group_fmriprep/hippocampus_posteriorR.nii.gz')

###### anterior

# one hemi
img = nib.load('masks/resampled/group_fmriprep/hippocampus.nii.gz')
mask = img.get_fdata()[:39,:,:]
ind = np.where(mask == 1)
indp = min(ind[1][:]) # check axis!
inda = max(ind[1][:]) # check axis!
third = int(np.floor((inda - indp) / 3))
mask_new = np.zeros(mask.shape, dtype = float)
mask_new[:, inda - third:inda + 1, :] = 1 # check axis!
ant1 = mask
ant1[mask_new == 0] = 0

# other hemi
img = nib.load('masks/resampled/group_fmriprep/hippocampus.nii.gz')
mask = img.get_fdata()[39:,:,:]
mask_new = np.zeros(mask.shape, dtype = float)
mask_new[:, inda - third:inda + 1, :] = 1 # check axis!
ant2 = mask
ant2[mask_new == 0] = 0

ant = np.concatenate((ant1,ant2))
img_ant = nib.Nifti2Image(ant, aff)
img_ant.to_filename('masks/resampled/group_fmriprep/hippocampus_anterior.nii.gz')
img_ant = nib.Nifti2Image(ant1, aff)
img_ant.to_filename('masks/resampled/group_fmriprep/hippocampus_anteriorL.nii.gz')
img_ant = nib.Nifti2Image(ant2, aff)
img_ant.to_filename('masks/resampled/group_fmriprep/hippocampus_anteriorR.nii.gz')

### Mentalizing network 

In [ ]:
# neurosynth
for dataversion in ['fmriprep']:
    denoised, analyses, wholebrain, parc_file = parc_files(dataversion)
    print(dataversion)
    group_mask = nib.load(wholebrain)
    group_maskf = group_mask.get_fdata().astype(bool)
    for roi in ['mentalizing']:

        cmask = nib.load('masks/resampled/other/neurosynth_' + roi + '.nii.gz')
        cmaskf = cmask.get_fdata()

        threshold = 2

        counts = np.unique(np.where(cmaskf > 0)[0], return_counts = True)
        counts_remove = counts[0][counts[1] < threshold]
        cmaskf[counts_remove,:,:] = 0
    
        counts = np.unique(np.where(cmaskf > 0)[1], return_counts = True)
        counts_remove = counts[0][counts[1] < threshold]
        cmaskf[:,counts_remove,:] = 0
    
        counts = np.unique(np.where(cmaskf > 0)[2], return_counts = True)
        counts_remove = counts[0][counts[1] < threshold]
        cmaskf[:,:,counts_remove] = 0
        
        cmask_group = cmaskf*group_maskf
        
        mask_nifti = nib.Nifti2Image(cmask_group.astype(np.float64),group_mask.affine)
        if dataversion == 'fmriprep':
            dataversion_save = 'group_fmriprep'
        nib.save(mask_nifti,f'masks/resampled/{dataversion_save}/neurosynth_{roi}.nii.gz')

In [ ]:
from scipy.ndimage import label

mentalizing = nib.load('masks/resampled/group_fmriprep/neurosynth_mentalizing.nii.gz')
mentalizingf = mentalizing.get_fdata().astype(bool)
mentalizingf = np.where(mentalizingf,1,0)

structure = np.ones((3, 3, 3))
labeled_array, num_clusters = label(mentalizingf, structure=structure)
cluster_masks = [(labeled_array == i).astype(int) for i in range(1, num_clusters + 1)]
cluster_masks_filt = [x for x in cluster_masks if len(x[x != 0]) > 100]

roi_labels = ['LTPJ','LTempPole','Precuneus','dmPFC','vmPFC','RTPJ','RTempPole']
#for i,label in enumerate(roi_labels):
#    roi = nib.Nifti2Image(cluster_masks_filt[i], mentalizing.affine, dtype = 'int64')
#    nib.save(roi,f'masks/resampled/group_fmriprep/ns_mentalizing_{label}.nii.gz')

In [ ]:
numbers = [1,2,3,4,5,1,2]
cluster_masks_filt_colored = []
for i,j in enumerate(cluster_masks_filt):
    cluster_masks_filt_colored.append(np.where(j != 0, j + numbers[i],j))

In [ ]:
from nilearn import plotting
# figure out what is what here
#view_img(nib.Nifti2Image(cluster_masks_filt[1] + cluster_masks_filt[6], mentalizing.affine, dtype = 'int64'), draw_cross = False)

#plotting.view_img(nib.Nifti2Image(np.stack(cluster_masks_filt_colored).sum(axis = 0), mentalizing.affine, dtype = 'int64'), draw_cross = False, cmap = 'hsv', symmetric_cmap = False)
plotting.plot_roi(nib.Nifti2Image(np.stack(cluster_masks_filt_colored).sum(axis = 0), mentalizing.affine, dtype = 'int64'), draw_cross = False, cmap = 'Reds', cut_coords = [-2,-54,14]).savefig('figs/paper/mentalizing_talk.jpg',dpi = 600)
#plotting.plot_roi(nib.Nifti2Image(np.stack(cluster_masks_filt_colored).sum(axis = 0), mentalizing.affine, dtype = 'int64'), draw_cross = False, cmap = 'Reds', black_bg = True,display_mode = 'y').savefig('figs/paper/mentalizing_talk.jpg',dpi = 600)

## Prep for statistical analyses

### Did the mentalizing network represent romantic interest?

Run `A1_RSA.py`. This file relies on a data file called `MV_DATA_ALL.pkl` which contains all of the fMRI data. If you'd like to see that file, please email me.

### How were neural representations of potential romantic partners updated in response to feedback?

Run `A2_RepChange.py`. This file relies on a data file called `MV_DATA_ALL.pkl` which contains all of the fMRI data. If you'd like to see that file, please email me.

### How often were neural representations of potential romantic partners reactivated during rest in response to feedback?

Run `A3_Reactivation.py`. This file relies on a data file called `MV_DATA_ALL.pkl` which contains all of the fMRI data. If you'd like to see that file, please email me.